### Purpose
This notebook is used to investigate the lower bound of the ratio of partial view size to dataset size: \alpha

In [1]:
import pandas as pd
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plot
import seaborn as sns
import scipy
import math
#from statsmodels.distributions.empirical_distribution import ECDF
from functools import reduce
from random import sample
# import dill
import os
%matplotlib inline
sns.set(style="ticks")

In [2]:
# Try to load the internal states (content of variables, etc.) from a previously saved session state
db_name = 'data/partial_view_no_rs.db'
try:
    dill.load_session(db_name) #Comment this line or delete the .db file if you want to start from a fresh state
except FileNotFoundError:
    print('No previous session state saved, skipping...')

No previous session state saved, skipping...


In [1]:
# Helper functions to detect and count known records

def isin_row(df, record, cols=None):
    cols = cols or df.columns
    return reduce(lambda x, y:x&y, [df[f].isin([record[f]]) for f in cols]).any()

def sample_records(df, records_count):
    n = df.shape[0]
    L = records_count
    known_records_indexes = sample(range(n), k=L)
    known_records = df.iloc[known_records_indexes,:]
    return (known_records_indexes, known_records)

def count_present_records(df, known_records):
    records_present = 0
    for i in range(len(known_records)):
        rec = known_records.iloc[i,:]
        if isin_row(df, rec):
            records_present += 1
    return records_present

def count_present_records_distinguish(df, known_records, known_records_indexes):
    records_occurrences = dict([])
    for i in range(len(known_records)):
        rec = known_records.iloc[i,:]
        records_occurrences[known_records_indexes[i]] = 0
        if isin_row(df, rec):
            records_occurrences[known_records_indexes[i]] += 1
    return records_occurrences

def check_pass_test(df, known_records):
    for i in range(len(known_records)):
        rec = known_records.iloc[i,:]
        if not isin_row(df, rec):
            return False
    return True

### Lower bound for \alpha
This lower bound arises from the false positives necessary condition $r_0 > 0$.

It is easy to see that $r_0 > 0 \iff P(R_{V,L} = 0) \leq 1-\eta$
so to get the absolute minimum $\alpha$ (to ensure $r_0 >= 1$) we end up with: $$\alpha_{min} = \min \left\{\alpha \in [1/n,1] : \frac{\prod_{k=0}^{\alpha*n -1}(n-k-L)}{\prod_{k=0}^{\alpha*n -1}(n-k)} \leq 1-\eta \right\}$$

that is $$\alpha_{min} = \min \left\{\alpha \in [1/n,1] : \frac{(n-L)(n-1-L)\cdots (n-\alpha*n + 1-L)}{n(n-1)\cdots (n- \alpha*n+1)} \leq 1-\eta \right\}$$

In [2]:
def get_alpha_min(r0, n, L, eta):
    '''Calculate minimum alpha to ensure a given r0'''
    alpha = 0
    while int(scipy.stats.hypergeom.isf(eta, n, L, alpha*n)) < r0:
        alpha += 1/n
    return alpha



In [21]:
2**25

33554432

In [3]:
def get_P0(n, Vmin, L):
    p0 = 1.0
    for k in range(Vmin): # k = 0,1,2...,V-1
        p0 *= (n-L-k)/(n-k)
    return p0



In [4]:
def get_smallest_alpha_min(n, L, eta):
    '''Calculate minimum alpha to ensure to ensure r0 >= 1'''
    V = 0
    alpha = 0
    while get_P0(n, V, L) >= 1-eta: # see math justification above
        V +=1
    alpha = V/n
    return alpha

In [8]:
n = 500000
eta = 0.95
L = 100

alpha_min = get_smallest_alpha_min(n, L, eta)
V_min = alpha_min*n
alpha_min, V_min

(0.029512, 14756.0)

In [6]:
n = 500000
eta = 0.95
L = 1000
r0=1

alpha_min = get_alpha_min(r0,n, L, eta)
V_min = alpha_min*n
alpha_min, V_min

(0.002989999999999947, 1494.9999999999736)

In [17]:
eta = 0.95
ns=[500000, 800000, 1000000]
L = 50
    
# alpha_min_090 = get_smallest_alpha_min(n, L, etas[0])
# alpha_min_095 = get_smallest_alpha_min(n, L, etas[1])
# alpha_min_099 = get_smallest_alpha_min(n, L, etas[2])


alpha_min_s = [get_smallest_alpha_min(n, L, eta) for n in ns]
ns, alpha_min_s



([500000, 800000, 1000000], [0.058154, 0.05815375, 0.058154])

In [18]:
eta = 0.95
ns=[500000, 800000, 1000000]
L = 10
    
# alpha_min_090 = get_smallest_alpha_min(n, L, etas[0])
# alpha_min_095 = get_smallest_alpha_min(n, L, etas[1])
# alpha_min_099 = get_smallest_alpha_min(n, L, etas[2])


alpha_min_s = [get_smallest_alpha_min(n, L, eta) for n in ns]
ns, alpha_min_s



([500000, 800000, 1000000], [0.258864, 0.258865, 0.258865])

In [2]:
0.26*500000, 0.26*800000, 0.26*1000000

(130000.0, 208000.0, 260000.0)

In [16]:
n = 500000
eta = 0.95
L = 1000

alpha_min = get_smallest_alpha_min(n, L, eta)
V_min = alpha_min*n
alpha_min, V_min

(0.00299, 1495.0)

In [26]:
n = 500000
eta = 0.95
L = 10

alpha_min = get_smallest_alpha_min(n, L, eta)
V_min = alpha_min*n
alpha_min, V_min

(0.258864, 129431.99999999999)

In [27]:
n = 500000
eta = 0.95
L = 1

alpha_min = get_smallest_alpha_min(n, L, eta)
V_min = alpha_min*n
alpha_min, V_min

KeyboardInterrupt: 

In [11]:
n = 500000
eta = 0.95
L = 1000

alpha_min = get_alpha_min(1, n, L, eta)
V_min = alpha_min*n
alpha_min, V_min


(0.002989999999999947, 1494.9999999999736)

In [15]:
n = 900000
eta = 0.95
L = 1000

alpha_min = get_alpha_min(1, n, L, eta)
V_min = alpha_min*n
alpha_min, V_min


(0.002989999999999839, 2690.9999999998554)

In [14]:
n = 1000000
eta = 0.95
# L = int(n*0.001)
L = 1000

alpha_min = get_smallest_alpha_min(n, L, eta)
V_min = alpha_min*n
alpha_min, V_min

(0.00299, 2990.0)

In [22]:
n = 1000000
etas=[0.90, 0.95, 0.99]
L = 1000

    
alpha_min_090 = get_smallest_alpha_min(n, L, etas[0])
alpha_min_095 = get_smallest_alpha_min(n, L, etas[1])
alpha_min_099 = get_smallest_alpha_min(n, L, etas[2])

#Lmins = [get_smallest_L_min(n, V, eta) for V in Vs]
# Vs[-1], Lmins[-1]

print('alpha_min090 =', alpha_min_090)
print('alpha_min095 =', alpha_min_095)
print('alpha_min099 =', alpha_min_099)




alpha_min090 = 0.002299
alpha_min095 = 0.00299
alpha_min099 = 0.004593


In [26]:
n = 5000000
etas=[0.90, 0.92, 0.94, 0.95, 0.96, 0.98, 0.99]
L = 1000
    
# alpha_min_090 = get_smallest_alpha_min(n, L, etas[0])
# alpha_min_095 = get_smallest_alpha_min(n, L, etas[1])
# alpha_min_099 = get_smallest_alpha_min(n, L, etas[2])


alpha_min_s = [get_smallest_alpha_min(n, L, eta) for eta in etas]
etas, alpha_min_s

# print('alpha_min090 =', alpha_min_090)
# print('alpha_min095 =', alpha_min_095)
# print('alpha_min099 =', alpha_min_099)


([0.9, 0.92, 0.94, 0.95, 0.96, 0.98, 0.99],
 [0.0022998, 0.0025224, 0.0028092, 0.002991, 0.0032134, 0.003904, 0.0045942])

In [32]:
n = 1000000
etas=[0.90, 0.92, 0.94, 0.95, 0.96, 0.98, 0.99]
L = 1000
    
# alpha_min_090 = get_smallest_alpha_min(n, L, etas[0])
# alpha_min_095 = get_smallest_alpha_min(n, L, etas[1])
# alpha_min_099 = get_smallest_alpha_min(n, L, etas[2])


alpha_min_s = [get_smallest_alpha_min(n, L, eta) for eta in etas]
etas, alpha_min_s

# print('alpha_min090 =', alpha_min_090)
# print('alpha_min095 =', alpha_min_095)
# print('alpha_min099 =', alpha_min_099)


([0.9, 0.92, 0.94, 0.95, 0.96, 0.98, 0.99],
 [0.002299, 0.002522, 0.002809, 0.00299, 0.003213, 0.003903, 0.004593])